# Onderzoeksvraag 2

### Doel
De tweede onderzoeksvraag die we willen beantwoorden luidt als volgt:

**In hoeverre kan de IMDB-score van een film worden voorspeld op basis van de sociale media-populariteit van de cast (zoals Facebook-likes) en de bechdeltest rating uit de externe dataset?**  
-De targetvariabele is de IMDB-score van de film  
-Featurevariabelen zijn Facebook-likes, en of de bechdeltest geslaagd is voor die film

### Modelkeuze

DecisiontreeClassifier kan niet, omdat de voorspellen waarde niet categorisch is maar numeriek

Regressie lijkt de beste optie te zijn

Todo

-importeren gecleande data  
-importeren en koppelen externe dataset  
-baseline model maken  
-hyperparameters aanpassen  
-conclusies en beantwoorden onderzoeksvraag


In [100]:
# imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [101]:
films = pd.read_csv('films.csv')
films.head()

,titel,directeur,directeur_likes,acteur_1,acteur_1_likes,acteur_2,acteur_2_likes,acteur_3,acteur_3_likes,cast_totaal_likes,film_likes,imdb_score,budget,omzet
0,Avatar,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,33000,7.9,237000000.0,7.605058e+08
1,Pirates of the Caribbean: At World's End,Gore Verbinski,563.0,Johnny Depp,40000.0,Orlando Bloom,5000.0,Jack Davenport,1000.0,48350,0,7.1,300000000.0,3.094042e+08
2,Spectre,Sam Mendes,0.0,Christoph Waltz,11000.0,Rory Kinnear,393.0,Stephanie Sigman,161.0,11700,85000,6.8,325850000.0,2.660987e+08
3,The Dark Knight Rises,Christopher Nolan,22000.0,Tom Hardy,27000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,23000.0,106759,164000,8.5,250000000.0,4.481306e+08
4,John Carter,Andrew Stanton,475.0,Daryl Sabara,640.0,Samantha Morton,632.0,Polly Walker,530.0,1873,24000,6.6,263700000.0,7.305868e+07


In [102]:
bechdel = pd.read_csv('bechdel.csv')
bechdel.head()
films['titel'] = films['titel'].str.rstrip()
print(films['titel'][0])
print(len(films['titel'][0]))



Avatar
6


In [103]:
data = pd.merge(films, bechdel, left_on='titel', right_on='title', how='inner')

In [104]:
data

,titel,directeur,directeur_likes,acteur_1,acteur_1_likes,acteur_2,acteur_2_likes,acteur_3,acteur_3_likes,cast_totaal_likes,film_likes,imdb_score,budget,omzet,title,rating,id,imdbid,year
0,Avatar,James Cameron,0.0,CCH Pounder,1000.0,Joel David Moore,936.0,Wes Studi,855.0,4834,33000,7.9,237000000.0,7.605058e+08,Avatar,3,582,499549.0,2009
1,Spectre,Sam Mendes,0.0,Christoph Waltz,11000.0,Rory Kinnear,393.0,Stephanie Sigman,161.0,11700,85000,6.8,325850000.0,2.660987e+08,Spectre,1,6519,2379713.0,2015
2,John Carter,Andrew Stanton,475.0,Daryl Sabara,640.0,Samantha Morton,632.0,Polly Walker,530.0,1873,24000,6.6,263700000.0,7.305868e+07,John Carter,3,3108,401729.0,2012
3,Spider-Man 3,Sam Raimi,0.0,J.K. Simmons,24000.0,James Franco,11000.0,Kirsten Dunst,4000.0,46055,0,6.2,258000000.0,3.365303e+08,Spider-Man 3,2,5483,413300.0,2007
4,Tangled,Nathan Greno,15.0,Brad Garrett,799.0,Donna Murphy,553.0,M.C. Gainey,284.0,2036,29000,7.8,260000000.0,2.008073e+08,Tangled,3,1782,398286.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1904,Slacker,Richard Linklater,0.0,Tommy Pallotta,5.0,Richard Linklater,0.0,Jean Caffeine,0.0,5,2000,7.1,23000.0,1.227508e+06,Slacker,3,1846,102943.0,1991
1905,Breaking Upwards,Daryl Wein,38.0,Zoe Lister-Jones,331.0,Heather Burns,212.0,Ebon Moss-Bachrach,211.0,1546,324,6.2,15000.0,7.638200e+04,Breaking Upwards,3,8593,1247644.0,2009
1906,Pink Flamingos,John Waters,0.0,Divine,462.0,Mink Stole,143.0,Edith Massey,105.0,760,0,6.1,10000.0,1.804830e+05,Pink Flamingos,3,708,69089.0,1972
1907,Primer,Shane Carruth,291.0,Shane Carruth,291.0,David Sullivan,45.0,Casey Gooden,8.0,368,19000,7.0,7000.0,4.247600e+05,Primer,1,1420,390384.0,2004


# Baseline model

In [105]:
# rating is 3 --> bechdeltest is gelsaagd
# rating is 0, 1 of 2 --> aantal Bechdel criteria waarvoor de film slaagt
# eerst zo proberen, converteren naar True/False als dit beter werkt

features = data[['film_likes', 'directeur_likes', 
        'acteur_1_likes', 'acteur_2_likes', 
        'acteur_3_likes', 'cast_totaal_likes', 'rating']]

target = data['imdb_score']

X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [106]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"r2 score: {r2}")

MSE: 0.9280210751784648
r2 score: 0.09634726142277261
